# 01. Event Graphs

This is visualization sample of ga4 events network powered by cytoscape.

## Install

In [1]:

# environment variables
%pip install python-dotenv
%load_ext dotenv
%dotenv

# code formatter
%pip install blackcellmagic
%load_ext blackcellmagic

# bigquery
%pip install google-cloud-bigquery google-cloud-bigquery-storage pyarrow tqdm pandas numpy
%load_ext google.cloud.bigquery

import ipywidgets as widgets
import ipycytoscape as ics
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Edges

In [2]:
%%bigquery df_edges

WITH

extracts AS (
  SELECT
    CAST(FLOOR(ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC) / 2) AS INT64) * 2 AS pair_odd_id,
    CAST(FLOOR((ROW_NUMBER() OVER (PARTITION BY user_pseudo_id ORDER BY event_timestamp ASC) + 1) / 2) AS INT64) * 2 AS pair_even_id,
    event_timestamp,
    user_pseudo_id AS user_id,
    event_name,
    traffic_source.medium AS traffic_source,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = 'page_title') AS page_title,
  FROM
    `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
  WHERE
    _TABLE_SUFFIX BETWEEN "20210101" AND "20210103"
    AND event_name NOT IN (
        "page_view",
        "session_start",
        "first_visit",
        "scroll",
        "user_engagement"
    )
),

prepares AS (
  SELECT
    *,
    STRUCT(
      -- reduce duplicated events
      event_name AS id,
      event_timestamp AS ts,
      traffic_source
    ) AS event,
  FROM
    extracts
),

aggregates AS (
  SELECT
    user_id,
    ARRAY_AGG(event ORDER BY event.ts ASC) AS events,
  FROM
    prepares
  GROUP BY
    user_id,
    pair_odd_id
  HAVING
    COUNT(*) = 2
  UNION ALL
  SELECT
    user_id,
    ARRAY_AGG(event ORDER BY event.ts ASC) AS events,
  FROM
    prepares
  GROUP BY
    user_id,
    pair_even_id
  HAVING
    COUNT(*) = 2
),

edges AS (
  SELECT
    user_id,
    events[OFFSET(0)].id AS source,
    events[OFFSET(1)].id AS target,
    events[OFFSET(1)].ts - events[OFFSET(0)].ts AS delta,
    -- meta
    events[OFFSET(1)].traffic_source AS traffic_source,
  FROM
    aggregates
  ORDER BY
    user_id ASC,
    events[OFFSET(0)].ts ASC
),

subs AS (
  SELECT
    source,
    target,
    traffic_source,
    AVG(delta) as sub_delta,
    COUNT(*) as sub_count,
  FROM
    edges
  GROUP BY
    source, target,
    -- meta
    traffic_source
)

SELECT
  source,
  target,
  AVG(sub_delta) as delta,
  SUM(sub_count) as cnt,
  ARRAY_AGG(
    STRUCT(
      traffic_source,
      sub_count as cnt,
      sub_delta as delta
    )
  ) AS sub,
FROM
  subs
WHERE
  source != target
GROUP BY
  source, target
-- reduce noisy edges
HAVING
  SUM(sub_count) > 10

Downloading: 100%|██████████| 28/28 [00:01<00:00, 17.47rows/s]


In [3]:
import datetime as dt
import math
import json

def map_to_edges():
    edges = list(
        map(
            lambda x: {
                "source": x[0],
                "target": x[1],
                "delta": str(dt.timedelta(seconds=math.floor(x[2] / 1000000))),
                "count": x[3],
                "subs": list(x[4]),
            },
            df_edges.values,
        )
    )
    counts = list(map(lambda x: x["count"], edges))
    counts_avg = np.mean(counts)
    counts_std = np.std(counts)
    for edge in edges:
        edge["width"] = max((edge["count"] - counts_avg) / counts_std * 5 + 2, 0.5)
        for sub in edge['subs']:
            sub['delta'] = str(dt.timedelta(seconds=math.floor(sub['delta'] / 1000000)))
    return edges

edges = map_to_edges()

## Nodes

In [4]:
def aggregate_edges_to_nodes():
    node_dict = {}
    for edge in edges:
        source = edge['source']
        target = edge['target']
        count = edge['count']
        if not source in node_dict:
            node_dict[source] = {"id": source, "count": 0, "goto": [], "come": []}
        if not target in node_dict:
            node_dict[target] = {"id": target, "count": 0, "goto": [], "come": []}
        node_dict[target]["count"] += count
        node_dict[source]["goto"].append(edge)
        node_dict[target]["come"].append(edge)
    nodes = list(node_dict.values())
    counts = list(map(lambda x: x["count"], nodes))
    counts_avg = np.mean(counts)
    counts_std = np.std(counts)
    for node in node_dict.values():
        node['size'] = (node["count"] - counts_avg) / counts_std * 30 + 50
    return nodes


nodes = aggregate_edges_to_nodes()

## Graphs

In [5]:
cytoscapeobj = ics.CytoscapeWidget()

In [6]:
cytoscapeobj.graph.add_graph_from_json(
    {
        "nodes": nodes,
        "edges": edges,
    },
    directed=True,
)

In [7]:
cytoscapeobj.set_style(
    [
        {
            "selector": "node",
            "css": {
                "content": "data(id)",
                "text-valign": "center",
                "color": "white",
                "text-outline-width": 1.5,
                "text-outline-color": "green",
                "background-color": "green",
                "width": 'data(size)',
                "height": 'data(size)',
            },
        },
        {
            "selector": "edge",
            "style": {
                "width": "data(width)",
                "label": "data(delta)",
                "font-size": "8.5px",
                "line-color": "#ccc",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
                "target-arrow-color": "green",
            },
        },
    ]
)
cytoscapeobj.set_layout(
    name="cola",
    animate=True,
    randomize=False,
    animationDuration=150000,
    nodeSpacing=55,
)

## Widgets

In [8]:
output_main = widgets.Output(layout={"width": "50%"})
output_meta = widgets.Output(layout={"width": "50%"})


def render_node_detail(event):
    data = event["data"]
    df_main = pd.DataFrame([{"id": data["id"], "count": data["count"]}])
    meta = data["goto"] + data["come"]
    for row in meta:
        del row["width"]
        del row["subs"]
    df_meta = pd.DataFrame(meta)

    with output_main:
        output_main.clear_output(wait=True)
        display(widgets.HTML("<strong>Node Info:</strong>"))
        display(df_main)
    with output_meta:
        output_meta.clear_output(wait=True)
        display(widgets.HTML("<strong>Edge Info:</strong>"))
        display(df_meta)


def render_edge_detail(event):
    data = event["data"]
    df_main = pd.DataFrame(
        [
            {
                "source": data["source"],
                "target": data["target"],
                "count": data["count"],
                "delta": data["delta"],
            }
        ]
    )
    df_meta = pd.DataFrame(data["subs"])

    with output_main:
        output_main.clear_output(wait=True)
        display(widgets.HTML("<strong>Edge Info:</strong>"))
        display(df_main)
    with output_meta:
        output_meta.clear_output(wait=True)
        display(widgets.HTML("<strong>Meta Info:</strong>"))
        display(df_meta)


cytoscapeobj.on("node", "click", render_node_detail)
cytoscapeobj.on("edge", "click", render_edge_detail)

In [9]:
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'cola', 'animate': True, 'randomize': False, 'animationDuration': 15…

In [10]:
widgets.HBox([output_main, output_meta])